To open on Google Colab\
https://colab.research.google.com/github/RodrigoAVargasHdz/CHEM-4PB3/blob/main/Course_Notes/Week5/gpytorch_h3o_pes.ipynb

In [ ]:
#install  GPytorch
!pip install gpytorch

## Homework!
Fit the H3O+ PES.

```python
#load data
data_url = "https://github.com/RodrigoAVargasHdz/CHEM-4PB3/raw/main/Course_Notes/data/h3o+"
data = pd.read_csv(data_url)
```

**References**:\
[Assessing Gaussian Process Regression and Permutationally Invariant Polynomial Approaches To Represent High-Dimensional Potential Energy Surfaces](https://pubs.acs.org/doi/10.1021/acs.jctc.8b00298)

Use a modern libraries,

1. https://docs.gpytorch.ai/en/stable/
2. https://gpjax.readthedocs.io/en/latest/
3. https://gpy.readthedocs.io/en/deploy/

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

#load data
data_url = "https://github.com/RodrigoAVargasHdz/CHEM-4PB3/raw/main/Course_Notes/data/h3o+"
data = pd.read_csv(data_url)
print(data.head)

Xtot = data.drop(['Energy'],axis=1).to_numpy()
ytot = data['Energy'].to_numpy()#[:,np.newaxis]

# data            
N = 500
Nval = 250 
X_train,X_test,y_train,y_test = train_test_split(Xtot,ytot,test_size=ytot.shape[0] - N,random_state=0)
X_test, y_test = X_test[:Nval],y_test[:Nval]
Xtr,Xtst,ytr,ytst = X_train,X_test,y_train,y_test


In [ ]:
import matplotlib
import matplotlib.pylab as plt

plt.figure(figsize=(10,7))
plt.hist(data['Energy'],bins=100,density=True)
plt.xlabel('Energy',fontsize=20)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)

## GPyTorch

In [ ]:
import torch
import gpytorch

# from numpy to torch
Xtr = torch.from_numpy(Xtr)
ytr = torch.from_numpy(ytr)
Xtst = torch.from_numpy(Xtst)
ytst = torch.from_numpy(ytst)

Xtot = torch.from_numpy(Xtot)
ytot = torch.from_numpy(ytot)

In [ ]:
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(
            gpytorch.kernels.RBFKernel(ard_num_dims=train_x.shape[1]))

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

In [ ]:
# initialize likelihood and model
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = ExactGPModel(Xtr, ytr, likelihood)

In [ ]:
# Find optimal model hyperparameters using ADAM

# Use the adam optimizer
# Includes GaussianLikelihood parameters
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

training_iter = 100
mll_trajectory = []
mll_trajectory_tst = []
for i in range(training_iter):
    model.train()
    likelihood.train()
    # Zero gradients from previous iteration
    optimizer.zero_grad()
    # Output from model
    output = model(Xtr)
    # Calc loss and backprop gradients
    loss = -mll(output, ytr)
    loss.backward()
    print('Iter %d/%d - Loss: %.3f  noise: %.6f' % (
        i + 1, training_iter, loss.item(),
        model.likelihood.noise.item()
    ))
    print('lengthscale: ', model.covar_module.base_kernel.lengthscale[0])
    mll_trajectory.append(loss.item())
    optimizer.step()
    with torch.no_grad(), gpytorch.settings.fast_pred_var():
        model.eval()
        likelihood.eval()
        ypred = likelihood(model(Xtst))
        mse = torch.sqrt(torch.sum((ypred.mean - ytst)**2))
        mll_trajectory_tst.append(mse)
        

In [ ]:
_,axs = plt.subplots(nrows=1,ncols=2,figsize=(10,10))
axs[0].plot(np.arange(training_iter), mll_trajectory,marker='o')
axs[1].plot(np.arange(training_iter), mll_trajectory_tst, marker='o')
axs[0].set_xlabel('Iterations',fontsize=18)
axs[0].set_ylabel('MLL',fontsize=18)
axs[1].set_xlabel('Iterations',fontsize=18)
axs[1].set_ylabel('RMSE', fontsize=18)


In [ ]:
# Prediction with GPyTorch
from torch.utils.data import TensorDataset, DataLoader

# 	In all other cases, he suggests using a power of 2 as the mini-batch size.
# 	So the minibatch should be 64, 128, 256, 512, or 1024 elements large.

dummy_test_y = torch.full_like(Xtot, dtype=torch.long, fill_value=0)
test_dataset = TensorDataset(Xtot, dummy_test_y)
test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False)

means = torch.tensor([0.])
stds = torch.tensor([[0.,0]])
with torch.no_grad():
	for x_batch, _ in test_loader:
		preds = likelihood(model(x_batch.double()))
		mean = preds.mean #.cpu()
		means = torch.cat([means, mean])
		l,u = preds.confidence_region()
		std = torch.column_stack((l,u))
		stds = torch.vstack((stds,std))
        # stds = torch.vstack(stds,std)


In [ ]:
from sklearn.metrics import r2_score

ytot_gp = means[1:].numpy() # torch to numpy 
r2 = r2_score(ytot_gp,ytot)

plt.figure(figsize=(10,8))
plt.scatter(ytot_gp,ytot,s=5)

low = np.min(np.stack((ytot_gp,ytot)).flatten())
high = np.max(np.stack((ytot_gp,ytot)).flatten())
plt.plot([low, high], [low, high], ls="--", c=".5", alpha=0.5)

plt.title('H3O+ PES (N=%s)'%N,fontsize=18)
plt.text(0.02,0.08,r'$R^{2}$ = %.3f'%r2,fontsize=18)
plt.xlabel('GP prediction',fontsize=18)
plt.ylabel('Quantum Chemistry',fontsize=18)
plt.show()